#### Example running SpaFHy with jupyter notebook

In [1]:
from model_driver import driver
from iotools import read_results
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os

#os.chdir(r'/projappl/project_2000908/nousu/SpaFHy_v2.0')
os.chdir(r'/Users/jpnousu/SpaFHy_v2.0')

# example of calling driver, reading results
outputfile = driver(create_ncf=True, create_spinup=False, folder='testcase_input')
results = xr.open_dataset(outputfile)

spatial_pcpy {'lai_conif': True, 'lai_decid': True, 'lai_grass': False, 'lai_shrub': False, 'hc': True, 'cf': True}
spatial_pbu {'org': True, 'root': True}
spatial_pspd {'deep_id': False}
/Users/jpnousu/SpaFHy_v2.0/testcase_input/hyytiala_32/site_main_class.asc
/Users/jpnousu/SpaFHy_v2.0/testcase_input/hyytiala_32/site_type_combined.asc
/Users/jpnousu/SpaFHy_v2.0/testcase_input/hyytiala_32/canopy_height.asc
/Users/jpnousu/SpaFHy_v2.0/testcase_input/hyytiala_32/canopy_fraction.asc
/Users/jpnousu/SpaFHy_v2.0/testcase_input/hyytiala_32/LAI_conif.asc
/Users/jpnousu/SpaFHy_v2.0/testcase_input/hyytiala_32/LAI_decid.asc


KeyError: 'cmask_bi'

In [ ]:
date = '1981-08-01'
plt.figure(figsize=(16,6))
ax=plt.subplot(1,2,1)
#results['bucket_psi_root'].sel(time=date).plot(cmap='coolwarm_r')
results['bucket_psi_root'].mean(dim=['time']).plot(vmin=-3, cmap='coolwarm_r')
plt.subplot(1,2,2, sharex=ax, sharey=ax)
#results['bucket_moisture_root'].sel(time=date)[20:-60,70:-100].plot(cmap='coolwarm_r')
results['bucket_moisture_root'].mean(dim=['time']).plot(cmap='coolwarm_r')
#results['bucket_moisture_root'].sel(time=date).plot(cmap='coolwarm_r')
#plt.subplot(1,2,3, sharex=ax, sharey=ax)
#plt.subplot(2,2,4, sharex=ax, sharey=ax)
#results['bucket_moisture_top'].sel(time=date)[20:-60,70:-100].plot(cmap='coolwarm_r')

In [ ]:
import xarray as xr
file = r'/scratch/project_2000908/nousu/SpaFHy_RESULTS/testcase_input_202403121454.nc'
results = xr.open_dataset(file)

In [ ]:
firedata = results[['forcing_wind_speed', 'forcing_air_temperature', 'forcing_relative_humidity', 
                    'bucket_psi_root']].copy()
results.close()

In [ ]:
# subset by month
firedatass = firedata.sel(time=firedata.time.dt.month.isin([5, 6, 7, 8, 9])).copy()
# subset by snow
#firedatass = firedata.sel(time=(results['canopy_snow_water_equivalent'].max(dim=['lat','lon']) == 0)).copy()

In [ ]:
firedatass['forcing_wind_speed'][:].plot()

In [ ]:
fp = r'/scratch/project_2000908/nousu/SpaFHy_RESULTS/spafhy_for_fire_3_24.nc'
firedatass.to_netcdf(fp, 'w')
firedatass.close()